In [1]:
from hops import hdfs
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.types import LongType
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
35,application_1558772277326_0007,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.

In [2]:
df = spark.read.format("parquet").load(hdfs.project_path() + "Resources/iot-benchmarks/data/topic-lwm2m-3303-temperature")

df.cache().count()

4187

In [3]:
df.printSchema()

root
 |-- measurement: struct (nullable = true)
 |    |-- timestamp: long (nullable = false)
 |    |-- endpointClientName: string (nullable = false)
 |    |-- instanceId: integer (nullable = false)
 |    |-- gatewayId: integer (nullable = false)
 |    |-- ipsoObject: struct (nullable = false)
 |    |    |-- sensorValue: double (nullable = false)
 |    |    |-- minMeasuredValue: double (nullable = true)
 |    |    |-- maxMeasuredValue: double (nullable = true)
 |    |    |-- minRangeValue: double (nullable = true)
 |    |    |-- maxRangeValue: double (nullable = true)
 |    |    |-- sensorUnits: string (nullable = true)
 |    |    |-- resetMinAndMaxMeasuredValues: boolean (nullable = true)
 |-- kafkaTimestamp: timestamp (nullable = true)
 |-- endpointClientName: string (nullable = true)

In [4]:
timeFmt = "yyyy-MM-dd HH:mm:ss.SSS"
sensDf = df \
    .select(F.to_timestamp(F.col('measurement.timestamp')/1000).alias('measurementTmp'), 'kafkaTimestamp', (F.col('measurement.timestamp')).alias('measurementTmpLong')) \
    .withColumn('kafkaTmpLong', F.substring_index('kafkaTimestamp', '.', -1).cast('float')) \
    .withColumn('kafkaTmpLong', F.when(F.col('kafkaTmpLong') < 100, F.col('kafkaTmpLong')*10).otherwise(F.col('kafkaTmpLong')).cast('long')) \
    .withColumn('kafkaTmpLong', (F.unix_timestamp('kafkaTimestamp', format=timeFmt)*1000 + F.col('kafkaTmpLong'))) \
    .withColumn("diffMilliseconds", (F.col('kafkaTmpLong') - F.col('measurementTmpLong')))
    

dates = ("2019-05-27",  "2019-05-28")
sensDf = sensDf.where(F.col('measurementTmp').between(*dates)).cache()

sensDf.count()

1809

In [5]:
sensDf.show(5, False)

+-----------------------+-----------------------+------------------+-------------+----------------+
|measurementTmp         |kafkaTimestamp         |measurementTmpLong|kafkaTmpLong |diffMilliseconds|
+-----------------------+-----------------------+------------------+-------------+----------------+
|2019-05-27 07:25:15.067|2019-05-27 07:25:15.428|1558941915067     |1558941915428|361             |
|2019-05-27 07:25:17.068|2019-05-27 07:25:17.429|1558941917068     |1558941917429|361             |
|2019-05-27 07:29:19.068|2019-05-27 07:29:19.429|1558942159068     |1558942159429|361             |
|2019-05-27 07:29:21.068|2019-05-27 07:29:21.427|1558942161068     |1558942161427|359             |
|2019-05-27 07:30:13.068|2019-05-27 07:30:13.428|1558942213068     |1558942213428|360             |
+-----------------------+-----------------------+------------------+-------------+----------------+
only showing top 5 rows

In [6]:
sensDf.printSchema()

root
 |-- measurementTmp: timestamp (nullable = true)
 |-- kafkaTimestamp: timestamp (nullable = true)
 |-- measurementTmpLong: long (nullable = true)
 |-- kafkaTmpLong: long (nullable = true)
 |-- diffMilliseconds: long (nullable = true)

In [7]:
sensDf.agg(F.avg(F.col("diffMilliseconds"))).show()

+---------------------+
|avg(diffMilliseconds)|
+---------------------+
|   358.43725815367605|
+---------------------+

In [8]:
sensDf.sort(F.col('measurementTmp')).show(10, False)

+-----------------------+-----------------------+------------------+-------------+----------------+
|measurementTmp         |kafkaTimestamp         |measurementTmpLong|kafkaTmpLong |diffMilliseconds|
+-----------------------+-----------------------+------------------+-------------+----------------+
|2019-05-27 07:17:09.075|2019-05-27 07:17:10.129|1558941429075     |1558941430129|1054            |
|2019-05-27 07:17:11.071|2019-05-27 07:17:11.444|1558941431071     |1558941431444|373             |
|2019-05-27 07:17:13.069|2019-05-27 07:17:13.455|1558941433069     |1558941433455|386             |
|2019-05-27 07:17:15.069|2019-05-27 07:17:15.44 |1558941435069     |1558941435440|371             |
|2019-05-27 07:17:17.07 |2019-05-27 07:17:17.445|1558941437070     |1558941437445|375             |
|2019-05-27 07:17:19.069|2019-05-27 07:17:19.436|1558941439069     |1558941439436|367             |
|2019-05-27 07:17:21.074|2019-05-27 07:17:21.445|1558941441074     |1558941441445|371             |


In [9]:
%%spark -o sensDf

In [10]:
%%local
sensDf

/srv/hops/anaconda/anaconda/envs/pname/lib/python3.6/site-packages/autovizwidget/widget/utils.py:50: FutureWarning:

A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.



Output()